<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Rozbudowa-i-optymalizacja-similarity/colabs/Optymalizacja_i_rozbudowa_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wnioski

Przetestowane zostały dwa podejścia liczenia similarity:
1.   podstawowe korzystające z pythonowego liczenia similarity na listach;
2.   alternatywne korzystające z biblioteki Pandas.

W trakcie testów podstawowe podejście okazało się znacznie szybsze ($\sim 3$ razy szybsze). Czas wykonywania pierwszej metody to $3-3.5$ minuty, a drugiej metody to ponad $10$ minut.

Wybrana (najszybsza) metoda `similarity_one_vs_all` została z kolei dodatkowo usprawniona i z czasu wykonywania wynoszącego od $3$ do $3.5$ minuty zdołaliśmy osiągnąć czas wykonywania w granicach od $1.3$ do $1.7$ minuty, co jest dużym (dwukrotnym) przyśpieszeniem szybkości działania metody.

Metody `similarity_one_vs_all` oraz funkcja rankingująca `get_ranking` zostały zaadoptowane do danych gotowych na "Checkpoint $3$" i ich zaktualizowane wersje to `similarity_one_vs_all_checkpoint3` oraz `get_ranking_checkpoint3`.



# Uwagi

W trakcie ładowania danych możemy zaznaczyć które kolumny są importowane (patrz kod w podrozdziale "Załadowanie danych z pliku o rozszerzeniu .parquet"), co potencjalnie zaoszczędza wykorzystywaną pamięć RAM w środowisku wykonwczym.

Metody liczące similairty korzystające z pythonowego podejścia do ich liczenia dostały opcję wpisania parametru `reduced_dataset` który domyślnie ustawiony jest na wartość `False`. Jeżeli decydujemy się na załadowanie jedynie niezbędnych do liczenia similarity kolumn, to parametr ten trzeba ustawić na wartość `True`. W przeciwnym przypadku funkcja licząca similarity będzie zwracała błąd i nie będzie się wykonywać.

# Przygotowanie notatnika

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1692, done.
remote: Counting objects: 100% (470/470), done.
remote: Compressing objects: 100% (244/244), done.
remote: Total 1692 (delta 342), reused 309 (delta 226), pack-reused 1222
Receiving objects: 100% (1692/1692), 6.05 MiB | 22.44 MiB/s, done.
Resolving deltas: 100% (1030/1030), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout Rozbudowa-i-optymalizacja-similarity

Branch 'Rozbudowa-i-optymalizacja-similarity' set up to track remote branch 'Rozbudowa-i-optymalizacja-similarity' from 'origin'.
Switched to a new branch 'Rozbudowa-i-optymalizacja-similarity'


## Załadowanie danych z pliku o rozszerzeniu .parquet

In [4]:
from lib.const import JOINED_DATA

W poradniku "Tworzenie skrotu do GDrive" w folderze tutorials jest poradnik jak sprawić żeby poniższa lokalizacja się poprawnie ładowała.

Za pomocą parametru `columns` możemy ustalić które kolumny mają zostać pobrane z naszego zbioru.


In [5]:
data = pd.read_parquet(JOINED_DATA, columns = ["nconst", "tconst", "titleType", "genres", "category"])
# TODO przepiąć się na nowszy plik gdy będzie on już zapisany na dysku

In [6]:
data.head(1)

,nconst,tconst,titleType,genres,category
0,nm0000004,"[tt0694438, tt0694654, tt0694932, tt0694936, t...","[tvEpisode, tvEpisode, tvEpisode, tvEpisode, t...","[Comedy, Music, Comedy, Music, Comedy, Music, ...",actor


In [7]:
sample = data.sample(n = 10_000, random_state=1)

# Podstawowa metoda liczenia similarity

In [8]:
from typing import List, Any, Tuple
import pandas as pd
import numpy

In [9]:
from lib.similarity_utils import *

Za pomocą metody `similarity_one_vs_all` liczymy podobieństwa pomiędzy jednym aktorem, a resztą aktorów z całego naszego zbioru danych. Najpierw znajdziemy $3$ aktorów najpodobniejszych do Sylvestera Stallone.


In [10]:
Arnold_schwarzenegger = prepare_pandas_row(find_actor(data, 'nm0000216'))

In [11]:
sample.head(3)

,nconst,tconst,titleType,genres,category
1350238,nm13012740,[tt15598948],[tvEpisode],[Drama],actress
41240,nm4907912,"[tt2613856, tt2248709, tt2332569]","[video, video, video]","[Comedy, Short, Comedy, Short, Comedy, Family,...",actor
382004,nm9396794,"[tt7600136, tt7600796]","[tvSeries, tvEpisode]","[Drama, Drama]",actor


Przy liczeniu similarity dla obciętego do niezbędnych kolumn zbioru danych, ustalamy parametr `reduced_dataset` w metodzie `similarity_one_vs_all` na `True`.

In [12]:
%%time
ids, values = similarity_one_vs_all(sample, Arnold_schwarzenegger, reduced_dataset=True)

CPU times: user 481 ms, sys: 70 ms, total: 551 ms
Wall time: 506 ms


In [13]:
values[:3]

[-0.92, -0.45999999999999996, -0.47]

# Alternatywna metoda liczenia similarity

Znajdujemy aktora i zostawiamy jego dane w postaci `pd.DataFrame`.

In [14]:
Arnold_schwarzenegger_pd = find_actor(data, 'nm0000216')

In [15]:
Arnold_schwarzenegger_pd

nconst                                               nm0000216
tconst       [tt0233469, tt0364056, tt0711516, tt15739442, ...
titleType    [movie, videoGame, tvEpisode, video, tvEpisode...
genres       [Action, Drama, Thriller, Action, Adventure, S...
category                                                 actor
Name: 1651659, dtype: object

## Test metody na próbce

In [17]:
%%time
ids_pd, values_pd = similarity_one_vs_all_pandas(sample, Arnold_schwarzenegger_pd)

CPU times: user 2.89 s, sys: 65.1 ms, total: 2.95 s
Wall time: 2.93 s


In [18]:
values_pd[:3]

[-0.92, -0.45999999999999996, -0.47]

# Testy i porównanie metod na podstawowych danych

In [19]:
%%time
ids1, val1 = similarity_one_vs_all(data, Arnold_schwarzenegger, reduced_dataset=True)

CPU times: user 1min 27s, sys: 1.37 s, total: 1min 29s
Wall time: 1min 29s


In [20]:
%%time
ids2, val2 = similarity_one_vs_all_pandas(data, Arnold_schwarzenegger_pd)

CPU times: user 10min 29s, sys: 3.14 s, total: 10min 32s
Wall time: 10min 36s
